### Import libraries

In [8]:
import numpy as np
import numpy_financial as npf
import csv

### Cutomization

In [9]:
# Input Parameters

WACC = 0.12                   # Weighted average cost of capital
E = 50.0                      # Equity in currency value
D = 50.0                      # Debt in currency value
tax_rate = 0.20               # Tax Rate in decimal
interest_rate = 0.08          # Debt Interest Rate in decimal
initial_investment = 100.0    # Initial Investment in currency value
n_years = 5                   # Project Duration in years
input_levered_cf = 30       # Levered CF in currency value

my_currency_unit="Million Dollars" 
my_output_csv="0702 cash_flow_details.csv"
my_output_doc="0702 computational_details.txt"

### Defining functions

In [10]:
# Helper Functions for Payback Periods

def compute_undiscounted_payback(cash_flows, initial_investment):
    cumulative = 0.0
    for i, cf in enumerate(cash_flows, start=1):
        cumulative += cf
        if cumulative >= initial_investment:
            fraction = (initial_investment - (cumulative - cf)) / cf
            return i - 1 + fraction
    return None

def compute_discounted_payback(cash_flows, discount_rate, initial_investment):
    cumulative = 0.0
    for i, cf in enumerate(cash_flows, start=1):
        discounted_cf = cf / ((1 + discount_rate) ** i)
        cumulative += discounted_cf
        if cumulative >= initial_investment:
            fraction = (initial_investment - (cumulative - discounted_cf)) / discounted_cf
            return i - 1 + fraction
    return None


### Main computation module 

In [11]:

# Compute Annual Tax Shield:
#    Tax Shield = D * interest_rate * tax_rate
annual_tax_shield = D * interest_rate * tax_rate  
annual_interest = D * interest_rate

# Derive Unlevered Operating Cash Flow:
#    Unlevered CF = Levered CF + interest - Tax Shield
unlevered_cf = input_levered_cf + annual_interest - annual_tax_shield  

# Unlevered Analysis (Derived from the Levered Input)
# Compute Unlevered Discount Rate using:
#r_unlevered = WACC * (E + D) / (E + D * (1 - tax_rate))
levered_fund_cost_equity = WACC * (E + D) - annual_interest
return_on_equity = levered_fund_cost_equity / E
r_unlevered=return_on_equity

# Compute Unlevered NPV:
npv_unlevered = sum([unlevered_cf / ((1 + r_unlevered) ** t) for t in range(1, n_years + 1)]) - initial_investment

#  Compute Unlevered IRR:
cash_flows_unlevered = [-initial_investment] + [unlevered_cf] * n_years
irr_unlevered = npf.irr(cash_flows_unlevered)

# Compute Payback Periods for Unlevered Cash Flows:
undiscounted_payback_unlevered = compute_undiscounted_payback([unlevered_cf] * n_years, initial_investment)
discounted_payback_unlevered = compute_discounted_payback([unlevered_cf] * n_years, r_unlevered, initial_investment)

# Levered Analysis (Using the Input Levered CF)
# Levered CF is the provided input value (input_levered_cf)
levered_cf = input_levered_cf

# Compute Levered NPV (Discounting at WACC):
npv_levered = sum([levered_cf / ((1 + WACC) ** t) for t in range(1, n_years + 1)]) - initial_investment

# Compute Levered IRR:
cash_flows_levered = [-initial_investment] + [levered_cf] * n_years
irr_levered = npf.irr(cash_flows_levered)

# Compute Payback Periods for Levered Cash Flows:
undiscounted_payback_levered = compute_undiscounted_payback([levered_cf] * n_years, initial_investment)
discounted_payback_levered = compute_discounted_payback([levered_cf] * n_years, WACC, initial_investment)

# Cash Flow Details (Year by Year) Including Initial Outflow
cash_flow_details = []

# Add Year 0 with the initial cash outflow for both derived unlevered and levered cases.
cash_flow_details.append([
    0,                       # Year 0
    -initial_investment,     # Derived Unlevered CF at Year 0 (initial outflow)
    1.0,                     # Discount Factor at Year 0
    -initial_investment,     # Discounted Derived Unlevered CF
    -initial_investment,     # Cumulative Discounted Derived Unlevered CF
    -initial_investment,     # Input Levered CF at Year 0 (initial outflow)
    1.0,                     # Discount Factor at Year 0 for levered
    -initial_investment,     # Discounted Input Levered CF
    -initial_investment      # Cumulative Discounted Input Levered CF
])
cumulative_discounted_unlev = -initial_investment
cumulative_discounted_lever = -initial_investment

for t in range(1, n_years + 1):
    # Derived Unlevered calculations (using derived unlevered_cf)
    disc_factor_unlev = 1 / ((1 + r_unlevered) ** t)
    discounted_unlev = unlevered_cf * disc_factor_unlev
    cumulative_discounted_unlev += discounted_unlev
    
    # Levered calculations (using provided levered_cf and discounting at WACC)
    disc_factor_lever = 1 / ((1 + WACC) ** t)
    discounted_lever = levered_cf * disc_factor_lever
    cumulative_discounted_lever += discounted_lever
    
    cash_flow_details.append([
        t,
        unlevered_cf,
        round(disc_factor_unlev, 4),
        round(discounted_unlev, 2),
        round(cumulative_discounted_unlev, 2),
        levered_cf,
        round(disc_factor_lever, 4),
        round(discounted_lever, 2),
        round(cumulative_discounted_lever, 2)
    ])


### Save cash flows   

In [12]:
# Save Cash Flow Details to a CSV File with UTF-8 Encoding
csv_filename = my_output_csv
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow([
        "Year", 
        "Derived Unlevered CF", 
        "Disc Factor (Unlev)", 
        "Discounted Unlev CF", 
        "Cumulative Disc Unlev CF",
        "Input Levered CF",
        "Disc Factor (Lever)",
        "Discounted Lever CF",
        "Cumulative Disc Lever CF"
    ])
    for row in cash_flow_details:
        writer.writerow(row)


### Save computational steps  

In [13]:
# Saving computational steps to a file with
doc_filename =my_output_doc 
with open(doc_filename, mode='w', encoding='utf-8') as doc_file:
    # Write header and input details
    doc_file.write("Computational Details Report\n")
    doc_file.write("========================================\n\n")
    doc_file.write("Input Parameters:\n")
    doc_file.write(f"  - WACC: {WACC:.2%}\n")
    doc_file.write(f"  - Equity: ${E} million\n")
    doc_file.write(f"  - Debt: ${D} million\n")
    doc_file.write(f"  - Tax Rate: {tax_rate:.2%}\n")
    doc_file.write(f"  - Debt Interest Rate: {interest_rate:.2%}\n")
    doc_file.write(f"  - Initial Investment: ${initial_investment} million\n")
    doc_file.write(f"  - Project Duration: {n_years} years\n")
    doc_file.write(f"  - Input Levered Operating CF: ${input_levered_cf} million per year\n")
    doc_file.write(f"  - Annual Tax Shield: ${annual_tax_shield:.2f} million per year\n")
    doc_file.write("  => Derived Unlevered Operating CF = Input Levered CF - Tax Shield = "
                   f"{input_levered_cf} - {annual_tax_shield:.2f} = {unlevered_cf:.2f} million per year\n\n")
    
# Unlevered Analysis Details
    doc_file.write("A. Unlevered Analysis (Derived):\n")
    doc_file.write("------------------------------\n")
    doc_file.write("1. Unlevered Discount Rate Calculation:\n")
    doc_file.write("   Formula: r_unlevered = (WACC * (E + D)) / (E + D * (1 - tax_rate))\n")
    doc_file.write(f"   Calculation: r_unlevered = ({WACC:.2f} * ({E} + {D})) / ({E} + {D}*(1 - {tax_rate}))\n")
    
    doc_file.write("2. Unlevered NPV Calculation:\n")
    doc_file.write("   Formula: NPV = Σ[Derived Unlevered CF / (1 + r_unlevered)^t] - Initial Investment, t = 1 to 5\n")
    for t in range(1, n_years + 1):
        discounted_value = unlevered_cf / ((1 + r_unlevered) ** t)
        doc_file.write(f"     Year {t}: CF = {unlevered_cf} / (1 + {r_unlevered:.4f})^{t} = {discounted_value:.2f}\n")
    sum_discounted_unlev = sum([unlevered_cf / ((1 + r_unlevered) ** t) for t in range(1, n_years + 1)])
    doc_file.write(f"   Sum of Discounted CFs = {sum_discounted_unlev:.2f}\n")
    doc_file.write(f"   Unlevered NPV = {sum_discounted_unlev:.2f} - {initial_investment} = {npv_unlevered:.2f}\n\n")
    
    doc_file.write("3. Unlevered IRR Calculation:\n")
    doc_file.write("   Cash Flow Series: " + str(cash_flows_unlevered) + "\n")
    doc_file.write(f"   Computed Unlevered IRR = {irr_unlevered:.2%}\n\n")
    
    doc_file.write("4. Unlevered Payback Periods:\n")
    doc_file.write(f"   Undiscounted Payback Period = {undiscounted_payback_unlevered:.2f} years\n")
    doc_file.write(f"   Discounted Payback Period (using r_unlevered) = {discounted_payback_unlevered:.2f} years\n\n")
    
    # Levered Analysis Details
    doc_file.write("B. Levered Analysis:\n")
    doc_file.write("--------------------\n")
    doc_file.write("1. Levered Cash Flow:\n")
    doc_file.write(f"   (Input) Levered CF is used directly: {input_levered_cf} {my_currency_unit}\n\n")
    
    doc_file.write("2. Levered NPV Calculation (Discounting at WACC):\n")
    doc_file.write("   Formula: NPV = Σ[Input Levered CF / (1 + WACC)^t] - Initial Investment, t = 1 to 5\n")
    for t in range(1, n_years + 1):
        discounted_lever_value = levered_cf / ((1 + WACC) ** t)
        doc_file.write(f"     Year {t}: CF = {levered_cf} / (1 + {WACC:.2f})^{t} = {discounted_lever_value:.2f}\n")
    sum_discounted_lever = sum([levered_cf / ((1 + WACC) ** t) for t in range(1, n_years + 1)])
    doc_file.write(f"   Sum of Discounted CFs = {sum_discounted_lever:.2f}\n")
    doc_file.write(f"   Levered NPV = {sum_discounted_lever:.2f} - {initial_investment} = {npv_levered:.2f}\n\n")
    
    doc_file.write("3. Levered IRR Calculation:\n")
    doc_file.write("   Cash Flow Series: " + str(cash_flows_levered) + "\n")
    doc_file.write(f"   Computed Levered IRR = {irr_levered:.2%}\n\n")
    
    doc_file.write("4. Levered Payback Periods:\n")
    doc_file.write(f"   Undiscounted Payback Period = {undiscounted_payback_levered:.2f} years\n")
    doc_file.write(f"   Discounted Payback Period (using WACC) = {discounted_payback_levered:.2f} years\n\n")
    
# Cash Flow Table Details
    doc_file.write("C. Yearly Cash Flow Details (also saved in 'cash_flow_details.csv'):\n")
    header = ("Year", "Derived Unlevered CF", "Disc Factor (Unlev)", "Discounted Unlev CF", "Cumulative Disc Unlev CF",
              "Input Levered CF", "Disc Factor (Lever)", "Discounted Lever CF", "Cumulative Disc Lever CF")
    doc_file.write("\t".join(header) + "\n")
    for row in cash_flow_details:
        row_str = "\t".join(str(item) for item in row)
        doc_file.write(row_str + "\n")
    
    doc_file.write("\nEnd of Computational Details Report.\n")

### Report Results to Console

In [14]:
# Reporting to console

print("Unlevered Analysis (Derived):")
print("-----------------------------")
print("Unlevered Discount Rate: {:.2%}".format(r_unlevered))
print("Unlevered NPV: {:.2f} {}".format(npv_unlevered, my_currency_unit))
print("Unlevered IRR: {:.2%}".format(irr_unlevered))
print("Unlevered Undiscounted Payback Period: {:.2f} years".format(undiscounted_payback_unlevered))
print("Unlevered Discounted Payback Period: {:.2f} years".format(discounted_payback_unlevered))

print("\nLevered Analysis (Input):")
print("-------------------------")
print("Annual Tax Shield: {:.2f} {} ".format(annual_tax_shield,my_currency_unit))
print("Levered NPV: {:.2f} {} ".format(npv_levered,my_currency_unit))
print("Levered IRR: {:.2%}".format(irr_levered))
print("Levered Undiscounted Payback Period: {:.2f} years".format(undiscounted_payback_levered))
print("Levered Discounted Payback Period: {:.2f} years".format(discounted_payback_levered))

print("\nCash flow details have been saved to '{}'".format(csv_filename))
print("Detailed computational steps have been saved to '{}'".format(doc_filename))

Unlevered Analysis (Derived):
-----------------------------
Unlevered Discount Rate: 16.00%
Unlevered NPV: 8.71 Million Dollars
Unlevered IRR: 19.68%
Unlevered Undiscounted Payback Period: 3.01 years
Unlevered Discounted Payback Period: 4.45 years

Levered Analysis (Input):
-------------------------
Annual Tax Shield: 0.80 Million Dollars 
Levered NPV: 8.14 Million Dollars 
Levered IRR: 15.24%
Levered Undiscounted Payback Period: 3.33 years
Levered Discounted Payback Period: 4.52 years

Cash flow details have been saved to '0702 cash_flow_details.csv'
Detailed computational steps have been saved to '0702 computational_details.txt'
